# Bayesian model-based hyperparameter optimization

In this notebook I will try an automated hyperparameter approach using Bayesian optimization.
Recent [results](http://proceedings.mlr.press/v28/bergstra13.pdf) suggest [Bayesian hyperparameter optimization](https://papers.nips.cc/paper/4522-practical-bayesian-optimization-of-machine-learning-algorithms.pdf) of machine learning models is more efficient than manual, random, or grid search with:

- Better overall performance on the test set
- Less time required for optimization


I will use the [HyperOpt](https://github.com/hyperopt/hyperopt) package because it is widely used.
Tor run the optimization we need the following:

- Objective function to minimize
- Space over which to search hyperparameters
- Algorithm for optimization
- History of results


In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
import hyperopt as hyper
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from timeit import default_timer as timer


In [ ]:
# Load and prepare dataframe for PO Netherlands
df = pd.read_csv('data/products_ordered_bq_sub_features.csv')
df = df.dropna()  # Drop future rows (no actuals)
df['actual'] = df['actual'] - df['blacklist']  # Remove blacklisted orders from target
df.drop('blacklist', axis=1, inplace=True)  # Drop blacklist feature
df.drop('key_id', axis=1, inplace=True)  # Drop key_id feature (we only have NL=1)
df.info()

# Define data to be used for training (use 3 years)
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] > '01-01-2016']
df['date'].describe()

# seasonality_dummy_columns = df.filter(regex=('^seasonality.*dummy.*$')).columns.values
# seasonality_dom_columns = df.filter(regex=('^seasonality\|day_of_month\|$')).columns.values
# holiday_columns = df.filter(regex=('.*holiday.*')).columns.values
# closed_day_columns = df.filter(regex=('.*closed.*')).columns.values
# training_columns = np.array([['date'], ['actual'], seasonality_dummy_columns,
#                              seasonality_dom_columns, holiday_columns, closed_day_columns])
# training_columns = [feature for array in training_columns for feature in array]  # Flatten list into single array
# df = df[training_columns]  # Subset data to defined columns


In [ ]:
# Prepare dataframes for modeling
features = [f for f in df.columns if f not in ['date', 'actual']]  # Define training features
X, y = df[features], df['actual']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)  # Split into train/test
X_train.shape, y_train.shape
X_test.shape, y_test.shape


In [ ]:
# Define objective function to minimize
def objective(params):
    """Objective function to optimize which is our XGBoost model"""
    start_time = timer() # Keep track of training time
    global ITERATION  # Keep track of evals
    ITERATION += 1

    dtrain = xgb.DMatrix(X_train, label=y_train)  # Create XGBoost dataframes
    dvalid = xgb.DMatrix(X_test, label=y_test)

    # Make sure parameters that need to be integers are integers
    for param_name in ['max_depth', 'subsample', 'min_child_weight']:
        params[param_name] = int(params[param_name])

    # params = {
    #     'objective': 'reg:linear',
    #     'learning_rate': 1,
    #     'max_depth': 2,
    #     'min_child_weight': 10,
    #     'subsample': 1,
    #     'gamma': 0.5,
    #     'colsample_bytree': 1,
    #     'eval_metric': 'rmse',
    #     'nthread': 4,
    #     'silent': 1}

    # watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    booster = xgb.train(params, dtrain)
    predictions = booster.predict(dvalid)

    loss = np.sqrt(mean_squared_error(y_test, predictions))  # Compute rmse of trail
    # print('rmse: {:.5}'.format(loss))

    run_time = timer() - start_time

    return {'loss': loss, 'params': params, 'iteration': ITERATION, 'train_time': run_time, 'status': hyper.STATUS_OK}


In [ ]:
# Define objective function to minimize
def optimize(bayes_trials, evals):
    """Define hyperparameter search space and optimize it using TPE"""
    search_space = {
        'boosting_type': 'gbdt',
        'num_boost_rounds' : hyper.hp.quniform('num_boost_rounds', 100, 1000, 1),
        'learning_rate': hyper.hp.quniform('learning_rate', 0.01, 0.5, 0.025),
        'max_depth': hyper.hp.quniform('max_depth', 1, 13, 1),
        'min_child_weight': hyper.hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hyper.hp.quniform('subsample', 0.7, 1, 0.5),
        'gamma': hyper.hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hyper.hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'rmse',
        'nthread': 4,
        'silent': 1,
        'verbose': -1}

    result = hyper.fmin(fn=objective, space=search_space, algo=hyper.tpe.suggest, trials=bayes_trials, max_evals=evals,
                        show_progressbar=True)


In [ ]:
# Run the optimization
ITERATION = 0
bayes_trials = hyper.Trials()  # Object where history of search trails are stored
optimize(bayes_trials, evals=10)  # Run Bayesian optimization


In [ ]:
# Check results
bayes_trials_results = sorted(bayes_trials.results, key = lambda x: x['loss'])  # Sort the results by loss
bayes_trials_results[1]  # Print results of optimal iteration